# 3. ADASYN
### 기존의 SMOTE는 모든 minority class로부터 동일한 개수의 synthetic 샘플을 생성했지만, ADASYN은 각 관측치마다 생성하는 샘플의 수가 다르다는 점이 특징이다. 일종의 weight로 synthetic 샘플 수를 결정하고 이 weight는 knn 범위 내로 들어오는 majority class의 개수에 비례하도록 한다. 논문에서 이렇게 설정하는 이유는 더 훈련시키기 어려운 관측치에 집중하여 근방의 synthetic 샘플을 더 많이 생성하는 것이라고 설명한다. 즉, 한 minority class의 knn 내 majority 개수가 많다면 훈련 시 majority class와 비슷한 설명변수를 갖는 해당 minority class를 majority로 분류할 가능성이 높아질텐데, 더 많은 샘플을 생성함으로 해당 minority class가 무시되지 않도록 하는 것이라고 해석해볼 수 있겠다.

In [1]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("./데이터자료/9-2. Std 데이터셋.csv")
feature = pd.read_csv("./데이터자료/8-2. Std feature.csv")
df

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,매출액증가율,유형자산증가율,재고자산증가율,...,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,label,회계년
0,(주)CMG제약,58820,2011-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,1.186494,-0.286555,-0.242712,...,-0.517775,-0.268715,-0.204982,-0.097181,0.001823,0.167352,-0.034998,-0.128958,0.0,2011
1,(주)CMG제약,58820,2012-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,-0.465320,-0.266139,-0.145700,...,-0.570848,-0.965957,-0.239707,-0.097246,-0.455154,7.331845,-8.395971,-8.842764,0.0,2012
2,(주)CMG제약,58820,2013-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.399443,-0.034331,0.368591,...,-0.459402,-0.280668,-0.160693,-0.097092,0.377872,-0.016097,0.127964,0.136844,0.0,2013
3,(주)CMG제약,58820,2014-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.042989,0.633335,-0.160104,...,-0.458301,-0.300589,-0.202938,-0.097179,0.294901,0.063259,0.057470,0.125748,0.0,2014
4,(주)CMG제약,58820,2015-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.229752,-0.280476,-0.110645,...,-0.503803,-0.558426,-0.217252,-0.097201,-0.090083,0.704439,-0.512105,-0.358349,0.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9273,휴마시스(주),205470,2020-12-01,2014-12-01,NaN,5,의료용 물질 및 의약품 제조업,5.662930,1.347455,2.510801,...,-0.151711,1.140567,-0.125778,-0.096931,0.930074,-0.576239,0.625551,0.569922,0.0,2020
9274,휴젤(주),145020,2019-12-01,2015-12-24,NaN,5,의료용 물질 및 의약품 제조업,0.059296,-0.046260,0.113775,...,1.299954,-0.272699,-0.062404,-0.097006,0.914756,-0.434150,0.499330,0.360994,0.0,2019
9275,휴젤(주),145020,2020-12-01,2015-12-24,NaN,5,의료용 물질 및 의약품 제조업,-0.170016,-0.100971,-0.437478,...,1.349171,-0.301727,-0.058107,-0.096969,0.907353,-0.415350,0.482630,0.330828,0.0,2020
9276,흥구석유(주),24060,2019-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,-0.488761,-0.299172,0.793390,...,-0.392823,-0.500939,-0.145811,-0.095817,-0.354313,-0.106535,0.208302,0.265758,0.0,2019


In [4]:
train = df[df["회계년"] <= 2018].reset_index(drop='index')
test  = df[df["회계년"] > 2018].reset_index(drop='index')
col = df.columns[7:55]

In [5]:
x_train = train[col]
y_train = train["label"]
x_test = test[col]
y_test = test["label"]
print(x_train.shape)
y_train.value_counts()

(6913, 48)


0.0    6792
1.0     121
Name: label, dtype: int64

In [6]:
adasyn = ADASYN(random_state=4)
x_train, y_train = adasyn.fit_resample(x_train,y_train)
print(x_train.shape)
y_train.value_counts()

(13553, 48)


0.0    6792
1.0    6761
Name: label, dtype: int64

In [7]:
mda_feature=feature["mda_feature"].tolist()
logit_feature=feature["logit_feature"].tolist()
lasso_feature=feature["lasso_feature"].tolist()
ela_feature=feature["ela_feature"].tolist()
same_feature=feature["same_feature"].tolist()

In [8]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [9]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")


In [10]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

In [11]:
mda_x_train = x_train[mda_feature].reset_index(drop='index')
mda_x_test = x_test[mda_feature].reset_index(drop='index')
mda_y_train = y_train
mda_y_test = test["label"]

In [12]:
logit_x_train = x_train[logit_feature].reset_index(drop='index')
logit_x_test = x_test[logit_feature].reset_index(drop='index')
logit_y_train = y_train
logit_y_test = test["label"]

In [13]:
lasso_x_train = x_train[lasso_feature].reset_index(drop='index')
lasso_x_test = x_test[lasso_feature].reset_index(drop='index')
lasso_y_train = y_train
lasso_y_test = test["label"]

In [14]:
ela_x_train = x_train[ela_feature].reset_index(drop='index')
ela_x_test = x_test[ela_feature].reset_index(drop='index')
ela_y_train = y_train
ela_y_test = test["label"]

In [15]:
same_x_train = x_train[same_feature].reset_index(drop='index')
same_x_test = x_test[same_feature].reset_index(drop='index')
same_y_train = y_train
same_y_test = test["label"]

In [16]:
fea_x_train = [mda_x_train, logit_x_train, lasso_x_train, ela_x_train, same_x_train]
fea_y_train = [mda_y_train, logit_y_train, lasso_y_train, ela_y_train, same_y_train]
fea_x_test = [mda_x_test, logit_x_test, lasso_x_test, ela_x_test, same_x_test]
fea_y_test = [mda_y_test, logit_y_test, lasso_y_test, ela_y_test, same_y_test]
fea = ["mda","logit","lasso","ela","same"]

In [17]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lda_clf.fit(x_train, y_train)
    lda_pred = lda_clf.predict(x_test)
    lda_pred_proba = lda_clf.predict_proba(x_test)
    print(feature,"lda") 
    get_clf_eval(y_test, lda_pred)
    get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)



mda lda
오차행렬
[[1837  478]
 [  15   35]]
정확도: 0.7915, 정밀도 : 0.0682, 재현율:0.7000,F1 스코어:0.1243
ROC 스코어: 0.7468, PR score : 0.0541
임곗값: 0.1
오차행렬
[[ 364 1951]
 [   4   46]]
정확도: 0.1734, 정밀도 : 0.0230, 재현율:0.9200,F1 스코어:0.0449
ROC 스코어: 0.5386, PR score : 0.0229
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[ 774 1541]
 [   8   42]]
정확도: 0.3450, 정밀도 : 0.0265, 재현율:0.8400,F1 스코어:0.0514
ROC 스코어: 0.5872, PR score : 0.0257
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1236 1079]
 [  10   40]]
정확도: 0.5395, 정밀도 : 0.0357, 재현율:0.8000,F1 스코어:0.0684
ROC 스코어: 0.6670, PR score : 0.0328
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1589  726]
 [  12   38]]
정확도: 0.6879, 정밀도 : 0.0497, 재현율:0.7600,F1 스코어:0.0934
ROC 스코어: 0.7232, PR score : 0.0429
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1837  478]
 [  15   35]]
정확도: 0.7915, 정밀도 : 0.0682, 재현율:0.7000,F1 스코어:0.1243
ROC 스코어: 0.7468, PR score : 0.0541
-

In [18]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    dt_clf.fit(x_train, y_train)
    dt_pred = dt_clf.predict(x_test)
    dt_pred_proba = dt_clf.predict_proba(x_test)   
    print(feature,"디시전트리") 
    get_clf_eval(y_test, dt_pred)
    get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 디시전트리
오차행렬
[[2116  199]
 [  37   13]]
정확도: 0.9002, 정밀도 : 0.0613, 재현율:0.2600,F1 스코어:0.0992
ROC 스코어: 0.5870, PR score : 0.0316
임곗값: 0.1
오차행렬
[[2116  199]
 [  37   13]]
정확도: 0.9002, 정밀도 : 0.0613, 재현율:0.2600,F1 스코어:0.0992
ROC 스코어: 0.5870, PR score : 0.0316
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2116  199]
 [  37   13]]
정확도: 0.9002, 정밀도 : 0.0613, 재현율:0.2600,F1 스코어:0.0992
ROC 스코어: 0.5870, PR score : 0.0316
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2116  199]
 [  37   13]]
정확도: 0.9002, 정밀도 : 0.0613, 재현율:0.2600,F1 스코어:0.0992
ROC 스코어: 0.5870, PR score : 0.0316
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2116  199]
 [  37   13]]
정확도: 0.9002, 정밀도 : 0.0613, 재현율:0.2600,F1 스코어:0.0992
ROC 스코어: 0.5870, PR score : 0.0316
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2116  199]
 [  37   13]]
정확도: 0.9002, 정밀도 : 0.0613, 재현율:0.2600,F1 스코어:0.0992
ROC 스코어: 0.5870, PR score : 0.0316

In [19]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    # key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
    param_grid = {
      'criterion': ['entropy', 'gini'],
               'max_depth': [2,4,6,8],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [4,6,8,10],
               'min_samples_split': [4,6,8,10]}
    grid_search = GridSearchCV(tree
                            , param_grid = param_grid
                            , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                            , n_jobs = -1)  #-1: 모든 코어 사용

    grid_search.fit(x_train, y_train)

    # 최선의 결과 출력
    print('피처:{2} 최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_, feature))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
    dt_clf_best.fit(x_train, y_train)
    dt_best_pred = dt_clf_best.predict(x_test)
    dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

    print(feature,"디시전트리 파라미터적용") 
    get_clf_eval(y_test, dt_best_pred)
    get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


피처:mda 최적 하이퍼 파라미터 :{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'auto', 'min_samples_leaf': 6, 'min_samples_split': 4}, 최적 평균 정확도 :0.820
mda 디시전트리 파라미터적용
오차행렬
[[2012  303]
 [  21   29]]
정확도: 0.8630, 정밀도 : 0.0873, 재현율:0.5800,F1 스코어:0.1518
ROC 스코어: 0.7246, PR score : 0.0595
임곗값: 0.1
오차행렬
[[ 987 1328]
 [  14   36]]
정확도: 0.4326, 정밀도 : 0.0264, 재현율:0.7200,F1 스코어:0.0509
ROC 스코어: 0.5732, PR score : 0.0249
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1547  768]
 [  17   33]]
정확도: 0.6681, 정밀도 : 0.0412, 재현율:0.6600,F1 스코어:0.0776
ROC 스코어: 0.6641, PR score : 0.0344
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1637  678]
 [  18   32]]
정확도: 0.7057, 정밀도 : 0.0451, 재현율:0.6400,F1 스코어:0.0842
ROC 스코어: 0.6736, PR score : 0.0365
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1977  338]
 [  21   29]]
정확도: 0.8482, 정밀도 : 0.0790, 재현율:0.5800,F1 스코어:0.1391
ROC 스코어: 0.7170, PR score : 0.0547
----------------------------

In [20]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    rf_clf.fit(x_train, y_train)
    rf_pred = rf_clf.predict(x_test)
    rf_pred_proba = rf_clf.predict_proba(x_test)
    print(feature,"랜덤포레스트") 
    get_clf_eval(y_test, rf_pred)
    get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 랜덤포레스트
오차행렬
[[2239   76]
 [  32   18]]
정확도: 0.9543, 정밀도 : 0.1915, 재현율:0.3600,F1 스코어:0.2500
ROC 스코어: 0.6636, PR score : 0.0825
임곗값: 0.1
오차행렬
[[1556  759]
 [  11   39]]
정확도: 0.6744, 정밀도 : 0.0489, 재현율:0.7800,F1 스코어:0.0920
ROC 스코어: 0.7261, PR score : 0.0428
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1926  389]
 [  22   28]]
정확도: 0.8262, 정밀도 : 0.0671, 재현율:0.5600,F1 스코어:0.1199
ROC 스코어: 0.6960, PR score : 0.0469
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2108  207]
 [  25   25]]
정확도: 0.9019, 정밀도 : 0.1078, 재현율:0.5000,F1 스코어:0.1773
ROC 스코어: 0.7053, PR score : 0.0645
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2187  128]
 [  28   22]]
정확도: 0.9340, 정밀도 : 0.1467, 재현율:0.4400,F1 스코어:0.2200
ROC 스코어: 0.6924, PR score : 0.0764
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2239   76]
 [  32   18]]
정확도: 0.9543, 정밀도 : 0.1915, 재현율:0.3600,F1 스코어:0.2500
ROC 스코어: 0.6636, PR score : 0.082

In [21]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    grid_search = {'criterion': ['entropy', 'gini'],
                'max_depth': [2,4,6,8],
                'max_features': ['auto', 'sqrt'],
                'min_samples_leaf': [4,6,8,10],
                'min_samples_split': [4,6,8,10],
                'n_estimators': [20,40,60,80,100]}

    model = RandomForestClassifier(random_state=52)
    grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                                cv = 5,verbose=0, n_jobs = -1)
    grid_search.fit(x_train,y_train)

    # 최선의 결과 출력
    print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
    rf_clf_best.fit(x_train, y_train)
    rf_best_pred = rf_clf_best.predict(x_test)
    rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

    print(feature,"디시전트리 파라미터적용") 
    get_clf_eval(y_test, rf_best_pred)
    get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 8, 'max_features': 'auto', 'min_samples_leaf': 6, 'min_samples_split': 4, 'n_estimators': 20}, 최적 평균 정확도 :0.852
mda 디시전트리 파라미터적용
오차행렬
[[2079  236]
 [  23   27]]
정확도: 0.8905, 정밀도 : 0.1027, 재현율:0.5400,F1 스코어:0.1725
ROC 스코어: 0.7190, PR score : 0.0652
임곗값: 0.1
오차행렬
[[ 408 1907]
 [   4   46]]
정확도: 0.1920, 정밀도 : 0.0236, 재현율:0.9200,F1 스코어:0.0459
ROC 스코어: 0.5481, PR score : 0.0234
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1439  876]
 [  10   40]]
정확도: 0.6254, 정밀도 : 0.0437, 재현율:0.8000,F1 스코어:0.0828
ROC 스코어: 0.7108, PR score : 0.0392
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1803  512]
 [  15   35]]
정확도: 0.7772, 정밀도 : 0.0640, 재현율:0.7000,F1 스코어:0.1173
ROC 스코어: 0.7394, PR score : 0.0511
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1950  365]
 [  18   32]]
정확도: 0.8381, 정밀도 : 0.0806, 재현율:0.6400,F1 스코어:0.1432
ROC 스코어: 0.7412, PR score : 0.0592
------------------

In [22]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    lr_pred_proba = lr_clf.predict_proba(x_test)
    print(feature,"로지스틱회귀") 
    get_clf_eval(y_test, lr_pred)
    get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

mda 로지스틱회귀
오차행렬
[[1833  482]
 [  16   34]]
정확도: 0.7894, 정밀도 : 0.0659, 재현율:0.6800,F1 스코어:0.1201
ROC 스코어: 0.7359, PR score : 0.0516
임곗값: 0.1
오차행렬
[[ 432 1883]
 [   4   46]]
정확도: 0.2021, 정밀도 : 0.0238, 재현율:0.9200,F1 스코어:0.0465
ROC 스코어: 0.5533, PR score : 0.0236
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[ 821 1494]
 [   8   42]]
정확도: 0.3649, 정밀도 : 0.0273, 재현율:0.8400,F1 스코어:0.0530
ROC 스코어: 0.5973, PR score : 0.0264
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1241 1074]
 [  12   38]]
정확도: 0.5408, 정밀도 : 0.0342, 재현율:0.7600,F1 스코어:0.0654
ROC 스코어: 0.6480, PR score : 0.0310
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1584  731]
 [  13   37]]
정확도: 0.6854, 정밀도 : 0.0482, 재현율:0.7400,F1 스코어:0.0905
ROC 스코어: 0.7121, PR score : 0.0411
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1833  482]
 [  16   34]]
정확도: 0.7894, 정밀도 : 0.0659, 재현율:0.6800,F1 스코어:0.1201
ROC 스코어: 0.7359, PR score : 0.051

In [23]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {'penalty': ['l2','l1','elasticnet','None'],
            'C':[0.01,0.1,1,3,5,10]}

    grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
    grid_search.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
    lr_clf_best.fit(x_train, y_train)
    lr_best_pred = lr_clf_best.predict(x_test)
    lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

    print(feature,"로지스틱회귀 파라미터적용") 
    get_clf_eval(y_test, lr_best_pred)
    get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미어:{'C': 0.01, 'penalty': 'l2'}, 최적 평균 정확도:0.775
mda 로지스틱회귀 파라미터적용
오차행렬
[[1839  476]
 [  16   34]]
정확도: 0.7920, 정밀도 : 0.0667, 재현율:0.6800,F1 스코어:0.1214
ROC 스코어: 0.7372, PR score : 0.0521
임곗값: 0.1
오차행렬
[[ 370 1945]
 [   4   46]]
정확도: 0.1759, 정밀도 : 0.0231, 재현율:0.9200,F1 스코어:0.0451
ROC 스코어: 0.5399, PR score : 0.0229
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[ 769 1546]
 [   7   43]]
정확도: 0.3433, 정밀도 : 0.0271, 재현율:0.8600,F1 스코어:0.0525
ROC 스코어: 0.5961, PR score : 0.0262
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1216 1099]
 [  11   39]]
정확도: 0.5307, 정밀도 : 0.0343, 재현율:0.7800,F1 스코어:0.0657
ROC 스코어: 0.6526, PR score : 0.0314
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1585  730]
 [  12   38]]
정확도: 0.6863, 정밀도 : 0.0495, 재현율:0.7600,F1 스코어:0.0929
ROC 스코어: 0.7223, PR score : 0.0427
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1839  476]
 [  16   34]]
정확도: 0.7920, 정밀도 : 0

In [24]:
# 1-5 KNN 
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    knn_clf.fit(x_train, y_train)
    knn_pred = knn_clf.predict(x_test)
    knn_pred_proba = knn_clf.predict_proba(x_test)
    print(feature,"KNN") 
    get_clf_eval(y_test, knn_pred)
    get_eval_by_threshold(y_test , knn_pred_proba[:,1].reshape(-1,1), thresholds)



mda KNN
오차행렬
[[2018  297]
 [  23   27]]
정확도: 0.8647, 정밀도 : 0.0833, 재현율:0.5400,F1 스코어:0.1444
ROC 스코어: 0.7059, PR score : 0.0547
임곗값: 0.1
오차행렬
[[1834  481]
 [  20   30]]
정확도: 0.7882, 정밀도 : 0.0587, 재현율:0.6000,F1 스코어:0.1070
ROC 스코어: 0.6961, PR score : 0.0437
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1929  386]
 [  21   29]]
정확도: 0.8279, 정밀도 : 0.0699, 재현율:0.5800,F1 스코어:0.1247
ROC 스코어: 0.7066, PR score : 0.0494
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1929  386]
 [  21   29]]
정확도: 0.8279, 정밀도 : 0.0699, 재현율:0.5800,F1 스코어:0.1247
ROC 스코어: 0.7066, PR score : 0.0494
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2018  297]
 [  23   27]]
정확도: 0.8647, 정밀도 : 0.0833, 재현율:0.5400,F1 스코어:0.1444
ROC 스코어: 0.7059, PR score : 0.0547
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2018  297]
 [  23   27]]
정확도: 0.8647, 정밀도 : 0.0833, 재현율:0.5400,F1 스코어:0.1444
ROC 스코어: 0.7059, PR score : 0.0547
-

In [25]:
# knn 파라미터 구하기
from sklearn.metrics import accuracy_score
acc = []
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    for k in range(1,20):
        knn_clf = KNeighborsClassifier(n_neighbors=k)
        knn_clf.fit(x_train, y_train)
        knn_pred = knn_clf.predict(x_test)
        knn_score=accuracy_score(y_test,knn_pred)

        acc.append(knn_score)
    int = acc.index(max(acc))+1
    print("최적거리")
    print(int)

    knn_clf_best =KNeighborsClassifier(n_neighbors=int)
    knn_clf_best.fit(x_train, y_train)
    knn_best_pred = knn_clf_best.predict(x_test)
    knn_best_pred_proba = knn_clf_best.predict_proba(x_test)   

    print(feature,"KNN 파라미터적용") 
    get_clf_eval(y_test, knn_best_pred)
    get_eval_by_threshold(y_test ,knn_best_pred_proba[:,1].reshape(-1,1), thresholds)      


최적거리
2
mda KNN 파라미터적용
오차행렬
[[2160  155]
 [  35   15]]
정확도: 0.9197, 정밀도 : 0.0882, 재현율:0.3000,F1 스코어:0.1364
ROC 스코어: 0.6165, PR score : 0.0413
임곗값: 0.1
오차행렬
[[2043  272]
 [  26   24]]
정확도: 0.8740, 정밀도 : 0.0811, 재현율:0.4800,F1 스코어:0.1387
ROC 스코어: 0.6813, PR score : 0.0499
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2043  272]
 [  26   24]]
정확도: 0.8740, 정밀도 : 0.0811, 재현율:0.4800,F1 스코어:0.1387
ROC 스코어: 0.6813, PR score : 0.0499
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2043  272]
 [  26   24]]
정확도: 0.8740, 정밀도 : 0.0811, 재현율:0.4800,F1 스코어:0.1387
ROC 스코어: 0.6813, PR score : 0.0499
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2043  272]
 [  26   24]]
정확도: 0.8740, 정밀도 : 0.0811, 재현율:0.4800,F1 스코어:0.1387
ROC 스코어: 0.6813, PR score : 0.0499
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2160  155]
 [  35   15]]
정확도: 0.9197, 정밀도 : 0.0882, 재현율:0.3000,F1 스코어:0.1364
ROC 스코어: 0.6165, PR sc

In [26]:
# 1-5 xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 

xgb_wrapper = XGBClassifier()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    xgb_wrapper.fit(x_train, y_train, verbose = True)
    xgb_wrapper.fit(x_train, y_train)
    xgb_pred = xgb_wrapper.predict(x_test)
    xgb_pred_proba = xgb_wrapper.predict_proba(x_test)
    print(feature,"XGB") 
    get_clf_eval(y_test, xgb_pred)
    get_eval_by_threshold(y_test , xgb_pred_proba[:,1].reshape(-1,1), thresholds)

mda XGB
오차행렬
[[2200  115]
 [  31   19]]
정확도: 0.9383, 정밀도 : 0.1418, 재현율:0.3800,F1 스코어:0.2065
ROC 스코어: 0.6652, PR score : 0.0670
임곗값: 0.1
오차행렬
[[2000  315]
 [  26   24]]
정확도: 0.8558, 정밀도 : 0.0708, 재현율:0.4800,F1 스코어:0.1234
ROC 스코어: 0.6720, PR score : 0.0450
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2096  219]
 [  28   22]]
정확도: 0.8956, 정밀도 : 0.0913, 재현율:0.4400,F1 스코어:0.1512
ROC 스코어: 0.6727, PR score : 0.0520
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2137  178]
 [  28   22]]
정확도: 0.9129, 정밀도 : 0.1100, 재현율:0.4400,F1 스코어:0.1760
ROC 스코어: 0.6816, PR score : 0.0602
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2176  139]
 [  30   20]]
정확도: 0.9285, 정밀도 : 0.1258, 재현율:0.4000,F1 스코어:0.1914
ROC 스코어: 0.6700, PR score : 0.0630
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2200  115]
 [  31   19]]
정확도: 0.9383, 정밀도 : 0.1418, 재현율:0.3800,F1 스코어:0.2065
ROC 스코어: 0.6652, PR score : 0.0670
-

In [27]:
values[1]

'l2'

In [28]:
#xgb 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

xgb_wrapper
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {
            'n_estimators':[40,43,46,50],
            'learning_rate':[0.03,0.05,0.04],
            'max_depth' : [0.8,1,2,3,4]
            }

    grid_clf = GridSearchCV(xgb_wrapper, param_grid=params, scoring='accuracy', cv=5)
    grid_clf.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

    a = grid_clf.best_params_
    keys=list(a.keys())
    values = list(a.values())
    xgb_clf_best = XGBClassifier( learning_rate=values[0],max_depth=values[1],n_estimators= values[2])
    xgb_clf_best.fit(x_train, y_train)
    xgb_best_pred = xgb_clf_best.predict(x_test)
    xgb_best_pred_proba = xgb_clf_best.predict_proba(x_test)   

    print(feature,"XGB 파라미터적용") 
    get_clf_eval(y_test, xgb_best_pred)
    get_eval_by_threshold(y_test ,xgb_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미어:{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 50}, 최적 평균 정확도:0.831
mda XGB 파라미터적용
오차행렬
[[1903  412]
 [  15   35]]
정확도: 0.8195, 정밀도 : 0.0783, 재현율:0.7000,F1 스코어:0.1408
ROC 스코어: 0.7610, PR score : 0.0612
임곗값: 0.1
오차행렬
[[ 298 2017]
 [   3   47]]
정확도: 0.1459, 정밀도 : 0.0228, 재현율:0.9400,F1 스코어:0.0445
ROC 스코어: 0.5344, PR score : 0.0227
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1181 1134]
 [   9   41]]
정확도: 0.5167, 정밀도 : 0.0349, 재현율:0.8200,F1 스코어:0.0669
ROC 스코어: 0.6651, PR score : 0.0324
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1665  650]
 [  11   39]]
정확도: 0.7205, 정밀도 : 0.0566, 재현율:0.7800,F1 스코어:0.1055
ROC 스코어: 0.7496, PR score : 0.0488
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1801  514]
 [  12   38]]
정확도: 0.7776, 정밀도 : 0.0688, 재현율:0.7600,F1 스코어:0.1262
ROC 스코어: 0.7690, PR score : 0.0574
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1903  412]
 [  15